<a href="https://colab.research.google.com/github/baseballwinpool/baseball/blob/master/2019_Baseball_Win_Pool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as p
import numpy as np

teams = p.read_csv('https://raw.githubusercontent.com/baseballwinpool/baseball/master/data/2019/2019teams.csv')
dfTeams = p.DataFrame(teams, index=None)
dfTeams.set_index('Code', inplace=True)

dictTeamsbyCode = dfTeams['FullName'].to_dict()
dictTeamsbyName = {v: k for k, v in dictTeamsbyCode.items()}


entries = p.read_csv('https://raw.githubusercontent.com/baseballwinpool/baseball/master/data/2019/2019picks.csv')
dfEntriesByPlayer = p.DataFrame(entries, index=None)
Players = dfEntriesByPlayer['User Name'].to_list()
dfEntriesByPlayer.set_index('User Name', inplace=True)


####
b = p.read_json('https://raw.githubusercontent.com/baseballwinpool/baseball/master/data/2019/2019entriesByPlayer.json')
###



tmp = p.read_csv('https://raw.githubusercontent.com/baseballwinpool/baseball/master/data/2019/2019picks.csv')
dfEntriesByTeam = p.DataFrame(tmp, index=None)
del dfEntriesByTeam['User Name']
dfEntriesByTeam = dfEntriesByTeam.T



#dfEntriesByTeam = dfEntriesByPlayer.T
dfEntriesByTeam.reset_index(inplace=True)
dfEntriesByTeam.rename(columns={'index': 'Team'}, inplace=True)
dfEntriesByTeam['Code'] = [dictTeamsbyName[x] for x in dfEntriesByTeam['Team']]

dfEntriesByTeam = dfEntriesByTeam.set_index('Code')
del dfEntriesByTeam['Team']

dfEntryStats = p.DataFrame(index=dfTeams.index)
dfEntryStats['AvgPick'] = np.round(dfEntriesByTeam.mean(axis=1), 1)
dfEntryStats.sort_values(inplace=True, ascending=False, by=['AvgPick'])
dfEntryStats['Rank'] = np.arange(1,31)

dictRankings = dfEntryStats['Rank'].to_dict()

dfStackedEntries = dfEntriesByTeam.stack().reset_index()
del dfStackedEntries['level_1']
dfStackedEntries.rename(columns={'Code' : 'Team', 0: 'Wins'}, inplace=True)
dfStackedEntries['Rank'] = [dictRankings[x] for x in dfStackedEntries['Team']]


rawStandings = p.read_json('https://erikberg.com/mlb/standings.json')
jsonstandings = rawStandings["standing"]

####
a = p.read_json('https://raw.githubusercontent.com/baseballwinpool/baseball/master/data/2019/2019stackedEntries.json')
###

teamsFromInternet = [jsonstandings[i]["first_name"] + " " + jsonstandings[i]["last_name"] for i in range (0,30)]
wins = [int(jsonstandings[i]["won"]) for i in range (0,30)]
losses = [int(jsonstandings[i]["lost"]) for i in range (0,30)]
gp = [int(jsonstandings[i]["games_played"]) for i in range (0,30)]

dfStandings = p.DataFrame(data=[teamsFromInternet, wins, losses, gp]).T
dfStandings.columns=('Team', 'Wins', 'Losses', 'GP')

dfStandings['Wins'] = dfStandings['Wins'].astype(int)
dfStandings['Losses'] = dfStandings['Losses'].astype(int)
dfStandings['GP'] = dfStandings['GP'].astype(int)
dfStandings['%GP'] = dfStandings['GP'] / float(1.62)
dfStandings['Code'] = [dictTeamsbyName[x] for x in dfStandings['Team']]
dfStandings = dfStandings.set_index('Code')
del dfStandings['Team']
dfStandings['OnPaceFor'] = np.round(dfStandings['Wins'] * 100 / dfStandings['%GP'], 1)
dfStandings = dfStandings.join(dfEntryStats['Rank'])


In [56]:
from bokeh.layouts import column
from bokeh.plotting import figure, show, output_notebook
#from bokeh.transform import jitter
from bokeh.models.widgets import Slider, Select, TextInput

player = 'Bulging Disk'
#playerSelect = Select(title="Player", options=["Pete", "Denver", "Bulging Disk"], value="Bulging Disk")
#playerSelect = Select(title="Player", options=Players)
#player1 = Select(title="Player", value="Bulging Disk")


ticks = sorted(dfEntryStats['Rank'].unique())


p1 = figure(plot_width=1200, plot_height=300, title="Win Predictions By Team: " + str(player))
p1.xgrid.grid_line_color = None
p1.xaxis[0].ticker = ticks

#Plot the entries
#rank = np.array(dfStackedEntries["Rank"]).astype(np.int)
#wins = np.array(dfStackedEntries["Wins"]).astype(np.int)
#p1.circle(rank, wins, size=9, alpha=0.4)
p1.circle(a["Rank"],a["Wins"], size=9, alpha=0.4)


#Plot the on pace for
p1.triangle(dfStandings['Rank'], dfStandings['OnPaceFor'], size=8, alpha=1, color='orange')

#def plotPlayer(attr, old, new):
#  abc = 17
  # insert callback function to plot player

#playerSelect.on_change('value', lambda attr, old, new: plotPlayer)

  
#Plot the picks for the selected player
#tmp = dfEntriesByPlayer.loc[dfEntriesByPlayer.index == player]
tmp = b.loc[b.index == player]
playerPicks = tmp.T
playerPicks.reset_index(inplace=True)
playerPicks.rename(columns={'index': 'Team', player:'Pick'}, inplace=True)
playerPicks['Code'] = [dictTeamsbyName[x] for x in playerPicks['Team']]
playerPicks['Rank'] = [dictRankings[x] for x in playerPicks['Code']]
p1.diamond(playerPicks['Rank'], playerPicks['Pick'], size=9, alpha=1, color='yellow')

invDictRankings = {v: k for k, v in dictRankings.items()}
p1.xaxis.major_label_overrides = invDictRankings

output_notebook()

#show(column(playerSelect,p1))
show(p1)

"{'HOU': 1, 'LAD': 2, 'NYY': 3, 'BOS': 4, 'CLE': 5, 'TB': 6, 'MIL': 7, 'NYM': 8, 'PHI': 9, 'ATL': 10, 'STL': 11, 'WAS': 12, 'MIN': 13, 'CHC': 14, 'OAK': 15, 'SEA': 16, 'LAA': 17, 'SD': 18, 'PIT': 19, 'AZ': 20, 'COL': 21, 'CIN': 22, 'TOR': 23, 'CWS': 24, 'DET': 25, 'SF': 26, 'TEX': 27, 'KC': 28, 'BAL': 29, 'MIA': 30}"